<a href="https://colab.research.google.com/github/alexsantostavares/TensorflowMLP/blob/master/TensorFlowMLPFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# garantir que seja instalado recurso utilizado
!pip install -U imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
#importando todas as bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler
import sys
import csv

In [27]:
#leitura do arquivo csv
df = pd.read_csv("/content/drive/MyDrive/Base/BD-2trab(CSV) (1).csv")

In [28]:
df

,Alternativo,Bar,Sex/Sab,Faminto,Cliente,Preço,Chove,Reserva,Tipo,Espera,Meta
0,Sim,Não,Não,Sim,Alg,$$$,Não,Sim,Fra,0-10,Sim
1,Sim,Não,Não,Sim,Che,$,Não,Não,Tai,30-60,Não
2,Não,Sim,Não,Não,Alg,$,Não,Não,Ham,0-10,Sim
3,Sim,Não,Sim,Sim,Che,$,Sim,Não,Tai,10-30,Sim
4,Sim,Não,Sim,Não,Che,$$$,Não,Sim,Fra,>60,Não
5,Não,Sim,Não,Sim,Alg,$$,Sim,Sim,Ita,0-10,Sim
6,Não,Sim,Não,Não,Ne,$,Sim,Não,Ham,0-10,Não
7,Não,Não,Não,Sim,Alg,$$,Sim,Sim,Tai,0-10,Sim
8,Não,Sim,Sim,Não,Che,$,Sim,Não,Ham,>60,Não
9,Sim,Sim,Sim,Sim,Che,$$$,Não,Sim,Ita,10-30,Não


In [29]:
# substituindo sim e não por 1 e 0
df = df.replace(['Sim'], 1)
df = df.replace(['Sim '], 1)
df = df.replace([' Sim'], 1)
df = df.replace(['sim'], 1)
df = df.replace(['sim '], 1)
df = df.replace([' sim'], 1)
df = df.replace(['Não'], 0)
df = df.replace(['Não '], 0)
df = df.replace([' Não'], 0)
df = df.replace(['não'], 0)
df = df.replace(['não '], 0)
df = df.replace([' não'], 0)


In [30]:
df.head()

,Alternativo,Bar,Sex/Sab,Faminto,Cliente,Preço,Chove,Reserva,Tipo,Espera,Meta
0,1,0,0,1,Alg,$$$,0,1,Fra,0-10,1
1,1,0,0,1,Che,$,0,0,Tai,30-60,0
2,0,1,0,0,Alg,$,0,0,Ham,0-10,1
3,1,0,1,1,Che,$,1,0,Tai,10-30,1
4,1,0,1,0,Che,$$$,0,1,Fra,>60,0


In [31]:
# converte string repetidas em valores numericos

palavras = {}

for coluna in df.columns:
  if type(coluna) is str:
    contador = df[coluna].unique()
    total = len(contador)
    mapeando = dict(zip(contador, range(total)))
    palavras[coluna] = mapeando
    df[coluna] = df[coluna].replace(mapeando)

In [32]:
df

,Alternativo,Bar,Sex/Sab,Faminto,Cliente,Preço,Chove,Reserva,Tipo,Espera,Meta
0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,1,0,1,1,1,1
2,1,1,0,1,0,1,0,1,2,0,0
3,0,0,1,0,1,1,1,1,1,2,0
4,0,0,1,1,1,0,0,0,0,3,1
5,1,1,0,0,0,2,1,0,3,0,0
6,1,1,0,1,2,1,1,1,2,0,1
7,1,0,0,0,0,2,1,0,1,0,0
8,1,1,1,1,1,1,1,1,2,3,1
9,0,1,1,0,1,0,0,0,3,2,1


In [33]:
# normalizando o formato de X e Y para que possam ser comparados
X = df[df.columns[:-1]].values
y = df[df.columns[-1]].values

In [34]:
# Mostrando o formato
X.shape, y.shape

((12, 10), (12,))

In [35]:
# normalizando o formato de X e Y para que possam ser comparados parte 2
scaler = StandardScaler()
X = scaler.fit_transform(X)
data = np.hstack((X, np.reshape(y, (-1, 1))))
transformed_df = pd.DataFrame(data, columns=df.columns)

In [36]:
# normalizando o formato de X e Y para que possam ser comparados parte 3
over = RandomOverSampler()
X, y = over.fit_resample(X, y)
data = np.hstack((X, np.reshape(y, (-1, 1))))
transformed_df = pd.DataFrame(data, columns=df.columns)

In [37]:
# dispozição dos valores do meta sim,não ou 1,0
len(transformed_df[transformed_df["Meta"]==1]), len(transformed_df[transformed_df["Meta"]==0])

(6, 6)

In [38]:
# determinando os treinamentos
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

In [39]:
# definindo a quantidade de camadas e metodo
model = tf.keras.Sequential([
                             tf.keras.layers.Dense(24, activation='relu'), # if x <= 0 --> 0, x > 0 --> x
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(16, activation='relu'),
                             tf.keras.layers.Dense(1, activation="sigmoid")
])

In [40]:
# mais alguns parametros e a taxa de aprendizagem
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [41]:
# pré avaliação com variaveis de treino
model.evaluate(X_train, y_train)

1/1 [==============================] - 0s 406ms/step - loss: 0.7376 - accuracy: 0.5714


[0.7376202940940857, 0.5714285969734192]

In [42]:
# pré avaliação 2
model.evaluate(X_valid, y_valid)

1/1 [==============================] - 0s 20ms/step - loss: 0.7848 - accuracy: 0.0000e+00


[0.784773588180542, 0.0]

In [43]:
# treinamento
model.fit(X_train, y_train, batch_size=8, epochs=40, validation_data=(X_valid, y_valid))

Epoch 1/40
1/1 [==============================] - 1s 673ms/step - loss: 0.7376 - accuracy: 0.5714 - val_loss: 0.7825 - val_accuracy: 0.0000e+00
Epoch 2/40
1/1 [==============================] - 0s 32ms/step - loss: 0.7260 - accuracy: 0.5714 - val_loss: 0.7798 - val_accuracy: 0.0000e+00
Epoch 3/40
1/1 [==============================] - 0s 36ms/step - loss: 0.7148 - accuracy: 0.5714 - val_loss: 0.7771 - val_accuracy: 0.0000e+00
Epoch 4/40
1/1 [==============================] - 0s 33ms/step - loss: 0.7037 - accuracy: 0.5714 - val_loss: 0.7745 - val_accuracy: 0.0000e+00
Epoch 5/40
1/1 [==============================] - 0s 32ms/step - loss: 0.6928 - accuracy: 0.5714 - val_loss: 0.7722 - val_accuracy: 0.0000e+00
Epoch 6/40
1/1 [==============================] - 0s 30ms/step - loss: 0.6827 - accuracy: 0.5714 - val_loss: 0.7701 - val_accuracy: 0.0000e+00
Epoch 7/40
1/1 [==============================] - 0s 60ms/step - loss: 0.6729 - accuracy: 0.5714 - val_loss: 0.7680 - val_accuracy: 0.0000e+0

In [44]:
# avaliação com as variaveis separadas para teste
model.evaluate(X_test, y_test)

1/1 [==============================] - 0s 30ms/step - loss: 0.8307 - accuracy: 0.3333


[0.8307296633720398, 0.3333333432674408]

In [45]:
# avaliação com todos os elementos
model.evaluate(X, y)

1/1 [==============================] - 0s 24ms/step - loss: 0.5787 - accuracy: 0.6667


[0.5786976218223572, 0.6666666865348816]